In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,Flatten
import glob
import pathlib
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
train = "../input/csc4851-homework4/birds_400/train"
test = "../input/csc4851-homework4/birds_400/test"
valid = "../input/csc4851-homework4/birds_400/valid"

In [3]:
def process(data):
    path=pathlib.Path(data)
    filepaths=list(path.glob(r"*/*.jpg"))
    labels=list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))
    df1=pd.Series(filepaths,name='filepaths').astype(str)
    df2=pd.Series(labels,name='labels')
    df=pd.concat([df1,df2],axis=1)
    return df

In [4]:
df_train=process(train)
df_test=process(test)
df_valid=process(valid)

In [5]:
train_generator = ImageDataGenerator( preprocessing_function=preprocess_input)
test_generator = ImageDataGenerator( preprocessing_function=preprocess_input)
valid_generator = ImageDataGenerator( preprocessing_function=preprocess_input)

In [6]:
train_image = train_generator.flow_from_dataframe(dataframe=df_train,
                                                x_col='filepaths',
                                                y_col='labels',
                                                target_size=(224,224),
                                                batch_size=16,
                                                subset='training',
                                                random_seed=42)

test_image = test_generator.flow_from_dataframe(
    dataframe=df_test,
    x_col='filepaths',
    y_col='labels',
    target_size=(224,224),
    batch_size=32
)

valid_image = test_generator.flow_from_dataframe(
    dataframe=df_valid,
    x_col='filepaths',
    y_col='labels',
    subset='training',
    target_size=(224,224),
    batch_size=32
)

In [14]:
pretrained_model = ResNet50(
    input_shape=(224,224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False

In [15]:
ip = pretrained_model.input

mod = Dense(120, activation='relu')(pretrained_model.output)
mod = Dense(120, activation='relu')(mod)

op = Dense(400, activation='softmax')(mod)

model = Model(inputs=ip, outputs=op)

In [16]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model_fit=model.fit(train_image,validation_data=valid_image,epochs=10)

In [20]:
pd.DataFrame(model_fit.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()
pd.DataFrame(model_fit.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()